# Chat Engine with CMEMRetriever (streaming)

To use the CMEM integration, use a RetrieverQueryEngine with CMEMRetriever and CMEMGraphStore.

In [1]:
%pip install cmem-cmempy llama-index python-dotenv

Note: you may need to restart the kernel to use updated packages.


Load environment from `.env` file. 
Start by `cp .env-template .env` and edit the content of `.env` accordingly.

In [2]:
%load_ext dotenv
%dotenv

Set up the LLM

In [3]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

model = "gpt-4o-mini"
llm = OpenAI(model=model)
Settings.llm = llm

Set up all components.

In [4]:
from llama_index.core import get_response_synthesizer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.core.memory import ChatMemoryBuffer

from llama_index_cmem.retrievers.cmem.cmem_retriever import CMEMRetriever

ontology_graph = "http://ld.company.org/prod-vocab/"
context_graph = "http://ld.company.org/prod-inst/"

response_synthesizer = get_response_synthesizer()

retriever = CMEMRetriever(ontology_graph=ontology_graph, context_graph=context_graph, llm=llm)

memory = ChatMemoryBuffer.from_defaults()

chat_engine = CondensePlusContextChatEngine(retriever=retriever, llm=llm, memory=memory)

Define a natural language query, run via query engine and get a natural language response answering the question.

In [5]:
from IPython.display import Markdown


def ask(prompt: str) -> None:
    """Ask a question and display the answer."""
    text = f"## Prompt: _{prompt}_\n\n"
    text += "### Final response:\n\n"
    display_handle = display(Markdown(""), display_id=True)
    display_handle.update(Markdown(text))
    response = chat_engine.stream_chat(prompt)
    for token in response.response_gen:
        text += f"{token}"
        display_handle.update(Markdown(text))


query = "List all hardware with price. Limit the results to 20 items."
ask(query)

## Prompt: _List all hardware with price. Limit the results to 20 items._

### Final response:

Here are 20 hardware items along with their prices:

1. **Hardware**: [hw-X716-6172862](http://ld.company.org/prod-instances/hw-X716-6172862) - **Price**: 3.2 EUR
2. **Hardware**: [hw-Q986-9996088](http://ld.company.org/prod-instances/hw-Q986-9996088) - **Price**: 1.12 EUR
3. **Hardware**: [hw-M605-5951566](http://ld.company.org/prod-instances/hw-M605-5951566) - **Price**: 3.94 EUR
4. **Hardware**: [hw-Y299-9772513](http://ld.company.org/prod-instances/hw-Y299-9772513) - **Price**: 0.38 EUR
5. **Hardware**: [hw-O184-6903943](http://ld.company.org/prod-instances/hw-O184-6903943) - **Price**: 1.15 EUR
6. **Hardware**: [hw-O311-4466005](http://ld.company.org/prod-instances/hw-O311-4466005) - **Price**: 3.1 EUR
7. **Hardware**: [hw-I980-1040313](http://ld.company.org/prod-instances/hw-I980-1040313) - **Price**: 3.38 EUR
8. **Hardware**: [hw-D762-3837218](http://ld.company.org/prod-instances/hw-D762-3837218) - **Price**: 2.43 EUR
9. **Hardware**: [hw-L128-5499058](http://ld.company.org/prod-instances/hw-L128-5499058) - **Price**: 0.51 EUR
10. **Hardware**: [hw-Q442-2123335](http://ld.company.org/prod-instances/hw-Q442-2123335) - **Price**: 1.65 EUR
11. **Hardware**: [hw-U286-4664935](http://ld.company.org/prod-instances/hw-U286-4664935) - **Price**: 0.38 EUR
12. **Hardware**: [hw-Z272-2955088](http://ld.company.org/prod-instances/hw-Z272-2955088) - **Price**: 0.82 EUR
13. **Hardware**: [hw-L536-5185541](http://ld.company.org/prod-instances/hw-L536-5185541) - **Price**: 4.22 EUR
14. **Hardware**: [hw-Z249-1364492](http://ld.company.org/prod-instances/hw-Z249-1364492) - **Price**: 4.12 EUR
15. **Hardware**: [hw-F344-7012314](http://ld.company.org/prod-instances/hw-F344-7012314) - **Price**: 3.25 EUR
16. **Hardware**: [hw-O857-5463957](http://ld.company.org/prod-instances/hw-O857-5463957) - **Price**: 3.62 EUR
17. **Hardware**: [hw-L586-5133830](http://ld.company.org/prod-instances/hw-L586-5133830) - **Price**: 3.08 EUR
18. **Hardware**: [hw-L316-4863597](http://ld.company.org/prod-instances/hw-L316-4863597) - **Price**: 1.23 EUR
19. **Hardware**: [hw-R860-6284767](http://ld.company.org/prod-instances/hw-R860-6284767) - **Price**: 1.02 EUR
20. **Hardware**: [hw-A688-6056899](http://ld.company.org/prod-instances/hw-A688-6056899) - **Price**: 0.98 EUR

Let me know if you need more information!